In [ ]:
import numpy as np
import pandas as pd
import emoji
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
!pip install emoji

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the data
mapping = pd.read_csv("/content/drive/MyDrive/internship/Mapping.csv")
output = pd.read_csv("/content/drive/MyDrive/internship/Test.csv")
train = pd.read_csv("/content/drive/MyDrive/internship/Train.csv")
test = pd.read_csv("/content/drive/MyDrive/internship/OutputFormat.csv")

In [ ]:
train

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3
...,...,...,...
69995,69995,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69996,69996,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69997,69997,If one of my daughters is wearing this and ask...,2
69998,69998,Guess who whoop people on THEIR homecoming?! #...,3


In [ ]:
#subset train DataFrame
subset_train=train.sample(frac=0.3).reset_index(drop=True)
subset_train.shape

(21000, 3)

In [ ]:
train

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3
...,...,...,...
69995,69995,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69996,69996,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69997,69997,If one of my daughters is wearing this and ask...,2
69998,69998,Guess who whoop people on THEIR homecoming?! #...,3


In [ ]:
# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['TEXT'])
vocab_size = len(tokenizer.word_index) + 1
train_sequences = tokenizer.texts_to_sequences(train['TEXT'])
test_sequences = tokenizer.texts_to_sequences(train['TEXT'])
max_seq_length = 100  # define the maximum sequence length
train_data = pad_sequences(train_sequences, maxlen=max_seq_length)
test_data = pad_sequences(test_sequences, maxlen=max_seq_length)


In [ ]:
# Check unique values in the label column
unique_labels = train['Label'].unique()
num_classes = len(unique_labels)


In [ ]:
# Preprocess labels
label_mapping = {label: index for index, label in enumerate(unique_labels)}
train_labels = train['Label'].map(label_mapping)
train_labels = to_categorical(train_labels, num_classes=num_classes)


In [ ]:
# Define the GRU model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_length))
model.add(GRU(128))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=10, batch_size=232)


Epoch 1/10
302/302 [==============================] - 221s 721ms/step - loss: 2.5138 - accuracy: 0.2516
Epoch 2/10
302/302 [==============================] - 213s 707ms/step - loss: 2.0430 - accuracy: 0.3850
Epoch 3/10
302/302 [==============================] - 213s 705ms/step - loss: 1.5017 - accuracy: 0.5530
Epoch 4/10
302/302 [==============================] - 221s 732ms/step - loss: 0.9722 - accuracy: 0.7177
Epoch 5/10
302/302 [==============================] - 213s 704ms/step - loss: 0.6457 - accuracy: 0.8130
Epoch 6/10
302/302 [==============================] - 220s 728ms/step - loss: 0.4687 - accuracy: 0.8621
Epoch 7/10
302/302 [==============================] - 212s 702ms/step - loss: 0.3679 - accuracy: 0.8902
Epoch 8/10
302/302 [==============================] - 211s 699ms/step - loss: 0.2998 - accuracy: 0.9093
Epoch 9/10
302/302 [==============================] - 211s 698ms/step - loss: 0.2520 - accuracy: 0.9247
Epoch 10/10
302/302 [==============================] - 210s 696m

In [ ]:
# Define a function for predicting emojis
def predict_emoji(input_sentence):
    # Preprocess the input sentence
    sequence = tokenizer.texts_to_sequences([input_sentence])
    data = pad_sequences(sequence, maxlen=max_seq_length)

    # Make prediction
    prediction = model.predict(data)
    predicted_label = np.argmax(prediction)

    # Map the predicted label to the emoji
    predicted_emoji = mapping['emoticons'][predicted_label]

    return predicted_emoji


In [ ]:
user_input = "i love deep learing "
predicted_emoji = predict_emoji(user_input)
print(f"Input Sentence: {user_input}")
print(f"Predicted Emoji: {predicted_emoji}")

1/1 [==============================] - 1s 632ms/step
Input Sentence: i love deep learing 
Predicted Emoji: 😂
